<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/leerazo/neo4j_aura_api/blob/main/03-deploying_aura/03-deploying_aura.ipynb" target="_blank">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/leerazo/neo4j_aura_api/blob/main/03-deploying_aura/03-deploying_aura.ipynb" target="_blank">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://github.com/leerazo/neo4j_aura_api/blob/main/03-deploying_aura/03-deploying_aura.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">Open in Vertex AI Workbench
    </a>
</td>
</table>


# Deploying Neo4j using the Aura API

Let's begin by installing and importing the needed python libraries.

This cell will automatically restart the kernel after installation.

If you see a message that "Your session crashed for an unknown reason" during this step, you can safely ignore that.

In [ ]:
%pip install python-dotenv
%pip install jupyterlab
%pip install ipywidgets

# Restart the kernel after installing libraries
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

In [ ]:
from dotenv import dotenv_values
from urllib.parse import urljoin
import ipywidgets as widgets
from IPython.display import display
import subprocess
import datetime
import json
import os

## Step 1: Authenticate API and get a bearer token for access

Upload the [API credential file you created earlier](../01-setup_aura_api/README.md) using the widget in the cell below.

In [ ]:
import sys
IN_COLAB = 'google.colab' in sys.modules
print('Google Colab:', str(IN_COLAB))

if IN_COLAB:
    from google.colab import files
    uploaded = files.upload()
    filename = next(iter(uploaded))
    content = uploaded[filename]
    cred_file = 'aura_api_creds.txt'
    os.rename(filename, cred_file)
else:
    cred_file = ''
    print('Currently only supported in Google Colab')

Now let's extract the credentials for further use. We'll use these to generate a bearer token and save both to files for later use.

In [ ]:
def refresh_token(api_creds, api_base):
    api_endpoint = urljoin(api_base, '/oauth/token')

    curl_cmd = "curl --request POST '{}' --user '{}:{}' --header 'Content-Type: application/x-www-form-urlencoded' --data-urlencode 'grant_type=client_credentials'".format(api_endpoint, api_creds['CLIENT_ID'], api_creds['CLIENT_SECRET'], api_creds['CLIENT_NAME'])
    result = json.loads(subprocess.check_output(curl_cmd, shell=True))
    access_token = result['access_token']
    expires_in = result['expires_in']

    now = datetime.datetime.now()
    expiration = (now + datetime.timedelta(0, expires_in)).isoformat()

    bearer_token = {
        'access_token': access_token,
        'expiration': expiration
    }

    # Save the bearer token to a file
    token_file = 'api_bearer_token'
    with open(token_file, "w") as outfile:
        json.dump(bearer_token, outfile, indent=4)

    access_token = bearer_token['access_token']
    expiration = bearer_token['expiration']
    return access_token, expiration

aura_creds = dotenv_values(cred_file)
api_base = 'https://api.neo4j.io/'
access_token, expiration = refresh_token(aura_creds, api_base)

And now we're ready to deploy our first Aura instance!


## Step 2: Choosing a cloud provider and configuring a new Aura instance.

Let's define a couple of helper functions.

In [ ]:
def list_tenants(access_token, api_base, tenant_id=None):
    #api_endpoint = 'https://api.neo4j.io/v1/tenants'

    api_endpoint = urljoin(api_base, '/v1/tenants')

    aura_tenants = {}

    if tenant_id:
        api_endpoint += '/' + tenant_id

    print()
    print('api_endpoint:', api_endpoint)
    print()

    list_cmd = "curl -s -X 'GET' '{}' -H 'accept: application/json' -H 'Authorization: Bearer {}'".format(api_endpoint, access_token)

    list_tenants = dict(json.loads(subprocess.check_output(list_cmd, shell=True)))['data']
    for item in list_tenants:
        aura_tenants[item['id']] = {}
        aura_tenants[item['id']]['id'] = item['id']
        aura_tenants[item['id']]['name'] = item['name']

    return aura_tenants


def tenant_info(access_token, api_base, tenant_id):
    api_endpoint = urljoin(api_base, '/v1/tenants/' + tenant_id)
    tenant_data = {}
    info_cmd = "curl -s -X 'GET' '{}' -H 'accept: application/json' -H 'Authorization: Bearer {}'".format(api_endpoint, access_token)
    tenant_info = json.loads(subprocess.check_output(info_cmd, shell=True))['data']

    for item in tenant_info:
        tenant_data[tenant_info['id']] = {}
        tenant_data[tenant_info['id']]['tenant_id'] = tenant_info['id']
        tenant_data[tenant_info['id']]['tenant_name'] = tenant_info['name']
        tenant_data[tenant_info['id']]['instance_configurations'] = list(tenant_info['instance_configurations'])

    return tenant_data



Now let's scan all of the tenants we have access to and choose from the list of available cloud providers using the dropdown menu in the widget below.

In [ ]:
aura_tenants = list_tenants(access_token, api_base)
tenant_ids = list(aura_tenants.keys())
available_configs = {}
print('aura_tenants:', aura_tenants)

config_list = []

available_clouds = []
available_types = []
available_versions = []
available_sizes = []

print()
print('Scanning available Aura tenants:')
print('--------------------------------')
for tenant_id in aura_tenants:
    print('tenant_id:', tenant_id, '"' + aura_tenants[tenant_id]['name'] + '"')
    #print('tenant_info:', aura_tenants[tenant_id])
    tenant_details = tenant_info(access_token, api_base, tenant_id)

    if tenant_id not in available_configs:
        available_configs[tenant_id] = {}
        available_configs[tenant_id]['tenant_name'] = tenant_details[tenant_id]['tenant_name']
        available_configs[tenant_id]['cloud_providers'] = {}

    for config in tenant_details[tenant_id]['instance_configurations']:
        config_dict = {
            'tenant_id': aura_tenants[tenant_id]['id'],
            'tenant_name': aura_tenants[tenant_id]['name'],
            'cloud_provider': config['cloud_provider'],
            'region': config['region'],
            'aura_type': config['type'],
            'neo4j_version': config['version'],
            'size': config['memory'],
            'size_int': int(config['memory'][0:len(config['memory'])-2])
        }
        config_list.append(config_dict)

        # Create a dictionary of all available configs
        if config['cloud_provider'] not in available_configs[tenant_id]['cloud_providers']:
            available_configs[tenant_id]['cloud_providers'][config['cloud_provider']] = {}
            available_configs[tenant_id]['cloud_providers'][config['cloud_provider']]['regions'] = {}
        if config['region'] not in available_configs[tenant_id]['cloud_providers'][config['cloud_provider']]['regions']:
            available_configs[tenant_id]['cloud_providers'][config['cloud_provider']]['regions'][config['region']] = {}
        if config['type'] not in available_configs[tenant_id]['cloud_providers'][config['cloud_provider']]['regions'][config['region']]:
            available_configs[tenant_id]['cloud_providers'][config['cloud_provider']]['regions'][config['region']][config['type']] = {}
        if config['version'] not in available_configs[tenant_id]['cloud_providers'][config['cloud_provider']]['regions'][config['region']][config['type']]:
            available_configs[tenant_id]['cloud_providers'][config['cloud_provider']]['regions'][config['region']][config['type']][config['version']] = []
        if config['memory'] not in available_configs[tenant_id]['cloud_providers'][config['cloud_provider']]['regions'][config['region']][config['type']][config['version']]:
            available_configs[tenant_id]['cloud_providers'][config['cloud_provider']]['regions'][config['region']][config['type']][config['version']].append(config['memory'])

        # Create a summary of all available configs
        if config['cloud_provider'] not in available_clouds:
            available_clouds.append(config['cloud_provider'])
        if config['type'] not in available_types:
            available_types.append(config['type'])
        if config['version'] not in available_versions:
            available_versions.append(config['version'])
        str_len = len(config['memory'])
        size_int = config['memory'][0:(str_len-2)]
        if int(size_int) not in available_sizes:
            str_len = len(config['memory'])
            size_int = config['memory'][0:(str_len-2)]
            available_sizes.append(int(size_int))
            available_sizes.sort()

#print('available_configs:')
#print(json.dumps(config_list, indent=4))

print('\nSummary:')
print('--------')
print('tenant_ids:', tenant_ids)
print('available_clouds:', sorted(available_clouds))
print('available_types:', sorted(available_types))
print('available_versions:', sorted(available_versions))
print('available_sizes:', available_sizes)

print('\nSelect one of the available cloud providers from the list below:')
selected_csp = widgets.Dropdown(
    options=available_clouds,
    disabled=False,
)

display(selected_csp)



Now run the cell below to select a desired Aura configuration using the dropdown menus in the widgets below.



In [ ]:
cloud_provider = selected_csp.value

available_tenants = []
available_types = []
available_versions = []
available_sizes = []

for config in config_list:
    if config['cloud_provider'] == cloud_provider:
        if config['tenant_id'] not in str(available_tenants):
            available_tenants.append(config['tenant_id'])
        if config['aura_type'] not in available_types:
            available_types.append(config['aura_type'])
        if config['neo4j_version'] not in available_versions:
            available_versions.append(config['neo4j_version'])
        if config['size_int'] not in available_sizes:
            available_sizes.append(config['size_int'])

available_types.sort()
available_versions.sort()
available_sizes.sort()

print('cloud_provider:', cloud_provider)
print('\nSelect configuration from dropdown menus below:\n')

selected_tenant = widgets.Dropdown(
    options=available_tenants,
    description='Tenant ID:',
    disabled=False,
)
selected_type = widgets.Dropdown(
    options=available_types,
    description='Aura type:',
    disabled=False,
)
selected_version = widgets.Dropdown(
    options=available_versions,
    description="Neo4j version:",
    disabled=False,
)
selected_size = widgets.Dropdown(
    options=available_sizes,
    description="Size (GB):",
    disabled=False,
)

print('Avaiable tenants:')
print('-----------------')
for tenant_id in available_tenants:
    print('tenant_id:', tenant_id, ' "' + tenant_info(access_token, api_base, tenant_id)[tenant_id]['tenant_name'] + '"')
print()

display(selected_tenant)
display(selected_type)
display(selected_version)
display(selected_size)

Let's define a couple more helper functions and we'll be ready to deploy.

In [ ]:
def deploy_instance(access_token, api_base, instance_name, tenant_id, cloud_provider, region='europe-west1', aura_type='professional-ds', neo4j_version='5', size='16GB'):
    api_endpoint = urljoin(api_base, '/v1/instances')

    instance_details = {}
    print('\nDeployment specifications:')
    print('neo4j_version:', neo4j_version)
    print('region:', region)
    print('memory:', size)
    print('name:', instance_name)
    print('type:', aura_type)
    print('tenant_id', tenant_id)
    print('cloud_provider:', cloud_provider)

    request_body = {
        "version": neo4j_version,
        "region": region,
        "memory": size,
        "name": instance_name,
        "type": aura_type,
        "tenant_id": tenant_id,
        "cloud_provider": cloud_provider
    }

    json_request_body = json.dumps(request_body)

    curl_cmd = "curl -s -X 'POST' '{}' -H 'accept: application/json' -H 'Authorization: Bearer {}' -H 'Content-Type: application/json'".format(api_endpoint, access_token)
    curl_cmd += " -d '{}'".format(json_request_body)

    api_response = json.loads(subprocess.check_output(curl_cmd, shell=True))

    response_data = api_response['data']

    instance_details['id'] = response_data['id']
    instance_details['name'] = response_data['name']
    instance_details['connection_url'] = response_data['connection_url']
    instance_details['password'] = response_data['password']
    instance_details['username'] = response_data['username']
    instance_details['cloud_provider'] = response_data['cloud_provider']
    instance_details['region'] = response_data['region']
    instance_details['tenant_id'] = response_data['tenant_id']
    instance_details['type'] = response_data['type']

    return instance_details

def list_instances(access_token, api_base, tenant_id=None):
    api_endpoint = urljoin(api_base, '/v1/instances')
    aura_instances = {}
    list_cmd = "curl -s -X 'GET' '{}' -H 'accept: application/json' -H 'Authorization: Bearer {}'".format(api_endpoint, access_token)

    list_instances = json.loads(subprocess.check_output(list_cmd, shell=True))['data']
    for instance in list_instances:
        aura_instances[instance['id']] = {}
        aura_instances[instance['id']]['instance_id'] = instance['id']
        aura_instances[instance['id']]['instance_name'] = instance['name']
        aura_instances[instance['id']]['cloud_provider'] = instance['cloud_provider']
        aura_instances[instance['id']]['tenant_id'] = instance['tenant_id']

    return aura_instances

def get_instance_info(access_token, api_base, instance_id):
    api_endpoint = urljoin(api_base, '/v1/instances/' + instance_id)
    instance_data = {}

    info_cmd = "curl -s -X 'GET' '{}' -H 'accept: application/json' -H 'Authorization: Bearer {}'".format(api_endpoint, access_token)

    api_response = json.loads(subprocess.check_output(info_cmd, shell=True))

    if 'data' in api_response:
        response_data = api_response['data']
        instance_status = response_data['status']

    return response_data

Great! Now run the cell below and get a list of all of the available regions supporting our specifications and give the new Aura instance a name in the widgets below.

In [ ]:
aura_type = selected_type.value
version = selected_version.value
size = str(selected_size.value) + 'GB'
tenant_id = selected_tenant.value

print('cloud_provider:', cloud_provider)
print('aura_type:', aura_type)
print('version:', version)
print('size:', size)
print('tenant_id:', tenant_id)

eligible_regions = []

for config in config_list:
    if (cloud_provider==config['cloud_provider']) and (tenant_id==config['tenant_id']) and (aura_type==config['aura_type']) and (version==config['neo4j_version']) and (size==config['size']):
        #eligible_configs.append({config['region'], config['tenant_name'], config['tenant_id']})
        if config['region'] not in eligible_regions:
            eligible_regions.append(config['region'])

#for row in eligible_configs:
#    print(row)
#    print(row['tenant_name'], row['region'])

selected_region = widgets.Dropdown(
    options=eligible_regions,
    description="Region:",
    disabled=False,
)
instance_name = widgets.Text(
    #value='Hello World',
    placeholder='Instance name',
    description='Neo4j instance name:',
    disabled=False
)

display(selected_region)
display(instance_name)

And now we're ready to go! Run the cell below to deploy our new instance. You can monitor its progress from the [Aura console](https://console.neo4j.io).

In [ ]:
new_name = instance_name.value
region = selected_region.value
aura_type = selected_type.value
neo4j_version = str(selected_version.value)
size = str(selected_size.value) + 'GB'

print('instance_name:', new_name)
print('tenant_id:', tenant_id)
print('cloud_provider:', cloud_provider)
print('region:', region)
print('aura_type:', aura_type)
print('neo4j_version:', neo4j_version)
print('size:', size)

instance_details = deploy_instance(access_token, api_base, new_name, tenant_id, cloud_provider, region=region, aura_type=aura_type, neo4j_version=neo4j_version, size=size)

print('\ninstance_details:')
for item in instance_details:
    print(item, '=', instance_details[item])

Let's take a closer look at our new instance. You may see some empty values while the instance is still in "creating" status. If so, wait a few minutes and run it again aftet it's finished deploying.

In [ ]:
instance_details = get_instance_info(access_token, api_base, instance_details['id'])
print('instance_details:')
print()
for item in instance_details:
    print('\t' + item, '=', instance_details[item])

## Congratulations!
That's it! You've deployed your first Aura instance using the Aura API.